In [2]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import math
from io import StringIO
import re

pca_evaluation_1_2 = pd.DataFrame({'Time Frame': [], 'View Weight': [], 'Ad Weight': [], 'Completion Weight': [], 
                                   'Average View Totals': [], 'Average Ad Totals': [], 'Average Completion Rate': []})

def run_pca(csv_file_pathway):
    # Get file name
    with open(csv_file_pathway, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    file_name = lines[0].strip().strip('"')

    # Load the data
    csv_content = '\n'.join(lines[3:])  # skip the first three rows
    df = pd.read_csv(StringIO(csv_content), delimiter='\t')

    # Clean up columns
    df.columns = (
        df.columns
        .str.strip()
        .str.replace(r'["\']', '', regex=True)
        .str.replace(r'[,\s]+$', '', regex=True)
    )
    
    # Define columns
    required_cols = [
        "CMS Url",
        "Videos Played--All Users",
        "Watched Ads--All Users",
        "Completion Rate--All Users"
    ]
    
    numeric_cols = [
        "Videos Played--All Users",
        "Watched Ads--All Users",
        "Completion Rate--All Users"
    ]
    
    # Check for missing columns
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in data: {missing_cols}")

    # Filter rows with required columns not null
    df_clean = df.dropna(subset=required_cols)
    if df_clean.empty:
        raise ValueError("No rows left after dropping rows with missing required values.")

    # Convert numeric columns to numbers
    for col in numeric_cols:
        df_clean[col] = (
            df_clean[col]
            .astype(str)
            .str.replace(r'[,"\']', '', regex=True)  # Remove commas, quotes
            .str.strip()
        )

        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # Drop rows with any NaNs in numeric columns
    df_clean = df_clean.dropna(subset=numeric_cols)
    if df_clean.empty:
        raise ValueError("No rows left after converting numeric columns and dropping NaNs.")

    # Assert all numeric columns are positive
    assert (df_clean[numeric_cols] >= 0).all().all(), "All scores should be positive"

    # Calculate scores
    df_clean["view_score"] = df_clean["Videos Played--All Users"] / (
        df_clean["Videos Played--All Users"].sum() / len(df_clean)
    )
    df_clean["ad_score"] = df_clean["Watched Ads--All Users"] / (
        df_clean["Watched Ads--All Users"].sum() / len(df_clean)
    )
    df_clean["completion_score"] = df_clean["Completion Rate--All Users"] / (
        df_clean["Completion Rate--All Users"].sum() / len(df_clean)
    )

    # Drop rows with any NaNs in score columns
    df_clean = df_clean.dropna(subset=['view_score', 'ad_score', 'completion_score']).copy()
    assert (df_clean[['view_score', 'ad_score', 'completion_score']] >= 0).all().all(), "All scores should be positive"

    # Standardize the scores
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_clean[['view_score', 'ad_score', 'completion_score']])

    # Apply PCA
    pca = PCA(n_components=1)
    df_clean['pca_score_raw'] = pca.fit_transform(X_scaled)

    # Normalize PCA output to center around 1
    mean_score = df_clean['pca_score_raw'].mean()
    df_clean['content_score'] = df_clean['pca_score_raw'] / mean_score

    # Adjust and extract component weights
    weight_sum = sum(pca.components_[0])
    weights_dict = {
        'Time Frame': file_name[31:],
        'View Weight': 0,
        'Ad Weight': 0,
        'Completion Weight': 0,
        'Average View Totals': df_clean["Videos Played--All Users"].sum() / len(df_clean), 
        'Average Ad Totals': df_clean["Watched Ads--All Users"].sum() / len(df_clean), 
        'Average Completion Rate': df_clean["Completion Rate--All Users"].sum() / len(df_clean)
    }
    for name, weight in zip(['view_score', 'ad_score', 'completion_score'], pca.components_[0]):
        weight = weight / weight_sum
        if name == 'view_score':
            weights_dict['View Weight'] = weight
        elif name == 'ad_score':
            weights_dict['Ad Weight'] = weight
        elif name == 'completion_score':
            weights_dict['Completion Weight'] = weight
    
    # Append the row to the DataFrame
    global pca_evaluation_1_2
    pca_evaluation_1_2 = pd.concat([pca_evaluation_1_2, pd.DataFrame([weights_dict])], ignore_index=True)
    
# Loop through years and months
for year, start_month, end_month in [
    (2023, 8, 12),   # 2023-08 to 2023-12
    (2024, 1, 6),   # 2024-01 to 2024-06
    (2025, 1, 5)     # 2025-01 to 2025-05
]: # Choose time range
    for month in range(start_month, end_month + 1):
        month_str = str(month).zfill(2)
        run_pca(f'/Users/parker.pape/Projects/Content Score 1.3/Data Table - Raw Content Score 1 Variables {year}-{month_str}.csv')

pca_evaluation_1_2.to_csv("/Users/parker.pape/Projects/Content Score 1.3/Video PCA 1.3 Output.csv", index=False)

print(pca_evaluation_1_2)

   Time Frame  View Weight  Ad Weight  Completion Weight  Average View Totals  \
0     2023-08     0.434612   0.434719           0.130669         48633.752177   
1     2023-09     0.450201   0.449985           0.099813         37682.772286   
2     2023-10     0.416282   0.415091           0.168628         26487.368548   
3     2023-11     0.401629   0.400870           0.197501         26181.550657   
4     2023-12     0.415253   0.414810           0.169937         28866.010547   
5     2024-01     0.414928   0.412950           0.172121         44877.945074   
6     2024-02     0.420049   0.417959           0.161992         28406.232558   
7     2024-03     0.404072   0.402740           0.193188         35694.552769   
8     2024-04     0.413612   0.412696           0.173692         43911.712598   
9     2024-05     0.409705   0.409505           0.180790         45951.782563   
10    2024-06     0.397633   0.396649           0.205718         44852.046067   
11    2025-01     0.411956  